# ADAMS Tutorial #4: Neural Networking (NN) Primer
## Part 2 of 3
This is the second of three notebooks which will cover our initial steps into creating our own neural network. The parts are as follows:

- Neural Network Structures and the Forward Pass
- Coding a Back Propagation
- Changes for Classification

## This notebook's topic: back propagation in neural networks ##
 1. Overview of the forward pass
 2. Caculating loss
 2. Finding a minimum
 3. Moving towards a minimum
 4. One solver for gradient descent: Stochastic gradient descent
 5. Stopping Rules
 6. Overview of back propagation

In [1]:
# Import relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import random

np.random.seed(888)  # set seed for reproducibility (numPy functions)
random.seed(888)  # set seed for reproducibility (random package functions)

range_for_demo = np.linspace(-5, 5, 100)

In [2]:
# Generate regression dataset
from sklearn.datasets import make_regression

n = 1000  # number of observations in our simulation
k = 15    # number of features in X in our simulation
h = 10    # number of hidden layer nodes

X, y = make_regression(n_samples=n, n_features=k, noise=5, random_state=888)

y = y.reshape(n, 1)  # Make sure that y is an array

So, we first went in detail through the elements of a neural network's forward pass in Part 2. Please read through that notebook and ensure that you understand its elements. We are now going to build on them for the next steps.

In [3]:
# Network architecture:
inputLayer_size = k     # number of features in X
hiddenLayer_size = h    # just a guess, will need experimentation to optimize
outputLayer_size = 1    # number of values to predict

In [4]:
# Weight initialization for first forward pass

limit = np.sqrt(6 / (inputLayer_size + outputLayer_size))  # Recommended weight initialization

weightsInputToHidden = np.random.uniform(-limit, limit, (hiddenLayer_size, inputLayer_size))    # Random weight initialization
weightsHiddenToOutput = np.random.uniform(-limit, limit, (outputLayer_size, hiddenLayer_size))  # Random weight initialization

biasInputToHidden  = np.ones((hiddenLayer_size,1))  # Bias initialization
biasHiddenToOutput = np.ones((outputLayer_size,1))  # Bias initialization

In [5]:
# Activation function that we will use in this scenario and its derivative
def ReLU(x):
    return np.maximum(0, x)  # 0 if input is negative, x if input is positive


def ReLU_derivative(x):
    return (ReLU(x) > 0).astype(int)

In [6]:
# First forward pass
inputs = np.array(X[5]).reshape((inputLayer_size, 1))  # observation 5
target = y[5].reshape((outputLayer_size, 1))  # true value of target
hiddenLayer_inputs = np.dot(weightsInputToHidden, inputs) + biasInputToHidden  # first hidden layer inputs
hiddenLayer_outputs = ReLU(hiddenLayer_inputs)  # hidden layer output (after activation)
outputLayer_inputs = np.dot(weightsHiddenToOutput, hiddenLayer_outputs) + biasHiddenToOutput  # input to output layer
outputLayer_outputs = ReLU(outputLayer_inputs)  # activation applied to output layer = prediction!
outputLayer_outputs.shape  # shape of first forward pass predictions

(1, 1)

Let's compare the loss and target for this observation. We can see that the random weight initialization is pretty far off.

In [7]:
print(outputLayer_outputs, target)

[[0.]] [[207.75432522]]


# Back Propagation

## Motivation:
With a set of random coefficients, would a prediction be any good? Probably not. However, the magic of neural networks comes from the method to update this complex system of weights. This is known as back propagation or backprop. 

## Calculating loss
The first step in back propagation is to calculate the loss (remember: loss is another term for error) which can be done in several ways. For **regression**, we would concentrate on mean square error, mean absolute error or even others such as mean square log error. There are also many other loss functions which act similarly to these. After computing the loss, we then want to minimize it.

Note that for some loss functions like the sigmoid for logistic regression, it is best to calculate the log likelihood. You can then derive this for maximization. 

In [8]:
# mean squared error
def mse(true, pred):
    sse = 0
    for i in range(len(true)):
        sse += (true[i] - pred[i])**2
    mse = (1 / len(true)) * np.sum(sse)
    return mse

For now, let's calculate the loss from our first model:

In [9]:
loss = mse(target, outputLayer_outputs)
loss

43161.859646772115

## Finding a minimum
Like a linear regression, we are working with input values along with layers of weights and a biases (= intercepts in the regression). Our loss function is a function of these three elements. Of these three elements, weights and bias are what we can adjust. So, let’s derive our loss function based on those. We will focus on deriving with respect to weights here, but the process for biases is similar.

Here is the mean square error. 
$$ MSE = \frac{1}{2n} \sum_{i=1}^n  (\hat y - y)^2 $$
Note that there is a 2 in the initial fraction. This is just for the convenience when we derive it later.

<img src="MSElossderivation.PNG" alt="Derivative of MSE" style="width: 600px;"/>

note that if $ w_0 $ is the intercept, we would set $ x=1 $.

Consider: Our loss function is a function of the true value and our NN’s output. Our NN’s output is a function of the last nodes’ outputs. Our last nodes’ outputs are all a function of their own input values, weights and biases. We’re lucky that we can simply use the chain rule to calculate the derivatives for each parameter! We will then have to organize our derivatives into gradients which are vectors of derivatives of different variables from the same function. For full optimization, we will need to calculate this derivative for every observation as well. We will discuss a simplification technique for this called stochastic gradient descent later.

So for each observation, we have to calculate: 

$$ \Bigg[ \frac{\partial MSE }{\partial w_0},  \frac{\partial MSE }{\partial w_1}, \frac{\partial MSE }{\partial w_2}, ... ,  \frac{\partial MSE }{\partial w_k} \Bigg] $$

In [10]:
def mse_derivative(true, pred, x):
    n = len(x)
    loss_d = (1/n) * np.sum((pred-true)*x)
    return loss_d.reshape(1, 1)


def ReLU_derivative(x):
    return (x > 0)*1.0

In [11]:
loss_derivative = mse_derivative(y[5], outputLayer_outputs, X[5])  # loss derivative
loss_derivative

array([[-34.98183822]])

In [12]:
activation_derivative = ReLU_derivative(outputLayer_inputs)  # activation derivative
activation_derivative

array([[0.]])

Remember that the output layer's output is a function of the following:
- the output layer's inputs
- the hidden layer's outputs
- the hidden layer's inputs
- the original X values

We will need to update all weights connecting these together.

In [13]:
# Update weights furthest back in the network (between hidden and output layer)
gradient_HiddenToOutput = np.dot(loss_derivative*activation_derivative, np.transpose(hiddenLayer_outputs))
gradient_HiddenToOutput.shape

# Update output layer biases
gradient_HiddenToOutput_bias = loss_derivative*activation_derivative

# Save the error of the output layer
outputLayer_errors = loss_derivative * activation_derivative

# Find gradient for next step for backpropagation: gradient to update weights between hidden and input layer
gradient_InputToHidden = np.dot(weightsHiddenToOutput.T, outputLayer_errors)
print(f'Shape of first weights {weightsHiddenToOutput.shape}')

# Next propagation backwards: derivative of the hidden layer output wrt the hidden layer input (ReLU derivative)
gradient_InputToHidden = gradient_InputToHidden * ReLU_derivative(hiddenLayer_inputs)
print(f'Shape of gradient vector {gradient_InputToHidden.shape}')

# Last propagation: derivate of the hidden layer input wrt to the weight matrix connecting the hidden layer to inputs X
gradient_InputToHidden = np.dot(gradient_InputToHidden, np.transpose(inputs))
print(f'Shape of input {inputs.shape}')

Shape of first weights (1, 10)
Shape of gradient vector (10, 1)
Shape of input (15, 1)


## Moving towards a minimum
We could update all weights by trying to take a large step down the gradient. However, since there are so many combinations of weights and bias, this just reach one local minimum of many, or it may overshoot the local minimum. It could be that there is a lower local minimum that we cannot yet identify. If we take smaller steps down the gradient, update weights/biases, make predictiongradient_HiddenToOutputs again with these new values and recalculate the gradient, we may find a steeper path which indicates the existence of a lower minimum. This step size is called a learning rate. This slow and iterative way of updating weights is called gradient descent.

$$ w_j' = w_j + lr \cdot \frac{\partial MSE }{\partial w_1} $$

Below are a few graphics. The one on the left shows the example of trying to fit a curve to points. The figure on the right is a depiction of slowly moving down a error surface to find a minimum.

In [14]:
%%html
<iframe src="https://giphy.com/embed/8tvzvXhB3wcmI" width="1000" height="400" frameBorder="0" class="giphy-embed" allowFullScreen></iframe>
<p><a href="https://giphy.com/gifs/deep-learning-8tvzvXhB3wcmI">via GIPHY</a></p>

We can come back to our scenario and begin the process of updating our weights based on our learning rate * derivative.

In [15]:
gradient_InputToHidden_bias = np.dot(weightsHiddenToOutput.T, outputLayer_errors) * ReLU_derivative(hiddenLayer_inputs)

In [16]:
# Gradient descent step
learningRate = 0.00001  # define some learning rate

# Update weights between hidden and output layer (furthest back)
weightsHiddenToOutput -= learningRate * gradient_HiddenToOutput
# Update bias in output layer
biasHiddenToOutput -= learningRate * gradient_HiddenToOutput_bias
# Update weights between input and hidden layer (furthest forward)
weightsInputToHidden -= learningRate * gradient_InputToHidden
# Update bias in hidden layer
biasInputToHidden -= learningRate * gradient_InputToHidden_bias

## One solver for gradient descent: SGD
As you can imagine, the number of computations for this process can be quite large. It requires one derivative per observation per parameter per step! Stochastic gradient descent (SGD) is one way to greatly simplify this task. This method picks samples randomly and only calculates their derivatives for optimization. By the strictest definition, SGD picks one sample per optimization step, but **it is common to optimize with batches of observations**. This could lead to a more stable path to a local minimum. In the code below, we will select a new variable batch_size, which will randomly select samples to optimize parameters.

## Stopping rules
What if we find the program is going through too many iterations with little improvement? We need to set a stopping rule to decide when the program should end. There are two very commonly chosen options: specifying a maximum number of epochs (= iterations) allowed or specifying a maximum allowed number of epochs without improvement in the accuracy of the validation set.

In [17]:
learningRate = 0.00001
epochs = 10  # stopping rule, how many corrective iterations we will allow
batch_size = 100

input_dim = X.shape[1]  # number of variables
output_dim = 1  # should be equal to 1 since we are only finding the predicted value for 1 regression

In [18]:
iteration = 0

loss_log = []

while iteration < epochs:

    # Put all the steps done before in a loop:
    # Process one observation per iteration
    squared_residuals = np.zeros(X.shape[0])  # to keep track of the loss

    random_obs = random.sample(range(0, len(X)), batch_size)  # randomly select a new sample of dataset for this iteration's optimization

    for obs in random_obs:

        inputs = np.array(X[obs]).reshape((input_dim, 1))  # select current case i
        target = y[obs].reshape((1, 1))  # and its target value

        # Compute the forward pass through the network all the way up to the final output
        hiddenLayer_inputs = np.dot(weightsInputToHidden, inputs) + biasInputToHidden
        hiddenLayer_outputs = ReLU(hiddenLayer_inputs)
        outputLayer_inputs = np.dot(weightsHiddenToOutput, hiddenLayer_outputs) + biasHiddenToOutput
        outputLayer_outputs = ReLU(outputLayer_inputs)  # final output

        # Compute the gradients:
        # gradient for the weights between hidden and output layers
        gradient_HiddenToOutput = mse_derivative(y[obs], outputLayer_outputs, X[obs]) * ReLU_derivative(outputLayer_outputs)
        outputLayer_errors = gradient_HiddenToOutput

        # gradient for the weights between input and hidden layers
        gradient_InputToHidden = np.dot(weightsHiddenToOutput.T, outputLayer_errors) * ReLU_derivative(hiddenLayer_inputs)

        # Perform gradient descent update
        biasHiddenToOutput -= learningRate * gradient_HiddenToOutput
        biasInputToHidden -= learningRate * gradient_InputToHidden

        weightsHiddenToOutput -= learningRate * np.dot(gradient_HiddenToOutput, np.transpose(hiddenLayer_inputs))
        weightsInputToHidden -= learningRate * np.dot(gradient_InputToHidden, np.transpose(inputs))

        # Store residual
        squared_residuals[obs] = np.sum((target-outputLayer_outputs)**2)

    inputs = np.array(X).reshape((inputLayer_size, n))  # all observations
    target = y.reshape((outputLayer_size, n))
    hiddenLayer_inputs = np.dot(weightsInputToHidden, inputs) + biasInputToHidden  # first hidden layer inputs
    hiddenLayer_outputs = ReLU(hiddenLayer_inputs)  # hidden layer output (after activation)
    outputLayer_inputs = np.dot(weightsHiddenToOutput, hiddenLayer_outputs) + biasHiddenToOutput  # input to output layer
    outputLayer_outputs = ReLU(outputLayer_inputs)
    iteration_loss_square = mse(target, outputLayer_outputs)
    iteration_loss = math.sqrt(iteration_loss_square)
    loss_log.append(iteration_loss)

    # Development of the loss as average over obs-level losses
    print(f'Epoch {iteration} with average error of {iteration_loss:.2f}.')
    iteration += 1

Epoch 0 with average error of 5481.54.
Epoch 1 with average error of 5481.23.
Epoch 2 with average error of 5480.74.
Epoch 3 with average error of 5479.89.
Epoch 4 with average error of 5478.98.
Epoch 5 with average error of 5477.78.
Epoch 6 with average error of 5476.90.
Epoch 7 with average error of 5476.01.
Epoch 8 with average error of 5475.04.
Epoch 9 with average error of 5473.83.


Look at that! As the epochs continue, we can see iterations show improvement in errors in the current random batch of observations. Important questions:
- Why could it be the case that the error is not continuously falling?
- What are some other good ideas for stopping rules for our algorithm? Or how can we manipulate the learning rate?
- Instead of looking at average loss per batch, could you think of a better way to measure model performance?

## Overview of back propagation
So, after setting up the neural network, we need to:
- Calculate loss from the forward pass
- Calculate the gradient of random observations (SGD)
- Update weights by moving down the gradient with a step size specified by your learning rate
- Make new predictions with the new weights
- Repeat the these steps until you reach a stopping rule